### A first try at classifying

Classifying is meant to be a relatively simple case of machine learning for coda.  We'll start by trying to find classifying data C such that C:odd and C:even have different logic if odd is data with an odd number of atoms and even is data with an even number of atoms. 

In [1]:
from base import *
from Space import Space
import Generate

In [2]:
#
#   First, we'll attempt to construct a search space which might contain a classifier as described above 
#
S = Generate.Gen(Generate.codas,3).space()

In [3]:
S

eval: False, true: 0, false:0, undecided:0, codas:143, datas:11568

In [4]:
for i in range(10): print(S[i])

{B ap} collect
{post B} pass
{B 2} rev
not or
{put B} some
{or B} tail
apby join
B {B 3}
1 {imply B}
* has1


In [5]:
Even = Generate.EvenAtoms(3)
Odd  = Generate.OddAtoms(3) 
Odd

eval: False, true: 0, false:0, undecided:0, codas:1, datas:3

In [6]:
#
#   The strategy is to first filter down S to the subspace which 
#   satisfies s:Even = ()... 
#
import Evaluate
def eval(d): return Evaluate.default(d)

def isempty (s,t): return eval(data(s|t)).empty()
def isatomic(s,t): return eval(data(s|t)).atomic()
                           
S_even = S.subop(isempty,Even)

In [7]:
S_even

eval: False, true: 0, false:0, undecided:0, codas:143, datas:1316

In [8]:
T = data(S_even[0]|Odd[2])

In [9]:
str(eval(T))

''

In [10]:
#
#   S_even is empty on even atoms up to 2 atoms.  Let's expand this to 20 atoms to clear it up. 
#
S_even2 = S_even.subop(isempty,Generate.EvenAtoms(20))

In [11]:
S_even2

eval: False, true: 0, false:0, undecided:0, codas:143, datas:1316

In [12]:
#
#   This step didn't do anything.  There are still 1316 candidate data 
#   satisfying d:X = () for X atoms up to 20. 
#
#   so... Now 
#
S_even_odd = S_even.subop(isatomic,Generate.OddAtoms(2))

In [13]:
S_even_odd

eval: False, true: 0, false:0, undecided:0, codas:4, datas:3

In [14]:
for d in S_even_odd: print(d)

apbin not
apbin =
apbin none


In [15]:
for d in S_even_odd: 
    for o in Generate.OddAtoms(3) : print(eval(data(d|o)),'<-',data(d|o))
for d in S_even_odd:
    for e in Generate.EvenAtoms(3): print(eval(data(d|e)),'<-',data(d|e))

◎ <- (apbin not:◎)
◎ <- (apbin not:◎ ◎ ◎)
◎ <- (apbin not:◎ ◎ ◎ ◎ ◎)
◎ <- (apbin =:◎)
◎ <- (apbin =:◎ ◎ ◎)
◎ <- (apbin =:◎ ◎ ◎ ◎ ◎)
◎ <- (apbin none:◎)
◎ <- (apbin none:◎ ◎ ◎)
◎ <- (apbin none:◎ ◎ ◎ ◎ ◎)
 <- (apbin not:)
 <- (apbin not:◎ ◎)
 <- (apbin not:◎ ◎ ◎ ◎)
 <- (apbin =:)
 <- (apbin =:◎ ◎)
 <- (apbin =:◎ ◎ ◎ ◎)
 <- (apbin none:)
 <- (apbin none:◎ ◎)
 <- (apbin none:◎ ◎ ◎ ◎)


### Success!  

This is my first attempt at "machine learning".  Notice that it works and it's doing something clever that I did not think of ahead of time.  `apbin not:X` combines the binary application `apbin` (used to translate a binary operator like a+b into a sequential sum), with a logical operation `not` to compute whether `X` has an even or odd number of atoms. 

It was only tested on the "hydrogen atom" (:), but it's now obvious that it works for any kind of atom as well as any number of atoms.  

```
step : apbin not : x y z w x 
[0] (({apbin not }:):({ x y z w x}:))
[1] (({apbin}:) ({not}:):({x}:) ({y z w x}:))
[2] (not x:(apbin not:y ({z}:) ({w x}:)))
[3] (not x:(not y:(not z:(apbin not:w x))))
[4] 𝝞𝟬𝟬𝝞𝟬𝝞𝝞𝝞𝟬𝟬𝝞𝝞𝝞𝟬
◎
```
* Isn't that clever?  

This is promising.  Compare trying to do this by searching strings interpreted as python programs to find one that distinguises even and odd number of arguments.  

Not only does this solve the immediate problem, but it provides additional insight and additional mathematical structure.  The insight is how to distinguish odd from even.  The additional content is that `apbin not` is a category.  
